## test code part 1

In [1]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chatbot import chat_model

In [2]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]

In [3]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which state Medicaid programs contract with private health insurance companies to provide healthcare services to Medicaid beneficiaries. In this model, the managed care organizations (MCOs) are responsible for coordinating and delivering a range of healthcare services, including preventive care, hospital services, and specialty care, to enrolled members.\n\nKey features of Medicaid managed care include:\n\n1. **Care Coordination**: MCOs help coordinate care among various providers to ensure that beneficiaries receive appropriate and timely services.\n\n2. **Cost Control**: By managing the care provided to beneficiaries, MCOs aim to control costs and improve efficiency in the delivery of healthcare services.\n\n3. **Access to Services**: Beneficiaries typically have a network of providers from which to choose, and MCOs may offer additional services beyond what traditional Medicaid covers.\n\n4. **Quality Improvement**: Many MCOs im

In [4]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_template.format(context=context, question=question)

"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n"

In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

## test code part 2

In [1]:
from langchain_intro.chatbot import review_chain

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e440f5f-9648-454b-b1c7-1f4e99719b65-0')

## test code 3

In [9]:
import dotenv
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from elasticsearch import Elasticsearch

dotenv.load_dotenv()

# Configuration for Elasticsearch
ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "reviews_index"

# Defining the embedding model
embeddings = HuggingFaceEmbeddings()

# Set up ElasticsearchStore
reviews_vector_db = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    embedding=embeddings,
    es_user="elastic",
    es_password="changeme",  # Replace with your password if security is enabled

)


# If you have documents to add, you can add them like this
# Assume `documents` is a list of Document objects
# documents = [Document(page_content="Some content", metadata={"source": "some source"})]
# reviews_vector_db.add_documents(documents)

# Question to search
question = """Has anyone complained about communication with the hospital staff?"""

# Perform similarity search
relevant_docs = reviews_vector_db.similarity_search(query=question, k=3)

# Display the relevant documents
print(relevant_docs[0].page_content)
print(relevant_docs[1].page_content)
print(relevant_docs[2].page_content)


I encountered some communication issues during my stay. The medical staff seemed disorganized, and it led to confusion about my treatment plan.
I encountered some issues with the nursing staff's communication. It seemed like there was a lack of coordination, leading to confusion about my medication schedule and treatment plan.
The hospital staff were friendly and attentive, making my stay more pleasant. However, there were occasional lapses in communication that caused confusion about my treatment plan.
